# Relaxation LDV Experiment

May 15th, 2020

Notebook by R. Vasudevan
and Kyle Kelley

NTF analysis on interferometric displacement sensing measurements using relaxation spectroscopy.


# Install and import necessary packages

In [ ]:
!pip install tensorly pyUSID

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyUSID as usid
import h5py
import os
from matplotlib.animation import FuncAnimation
import tensorly as tl
from tensorly.decomposition import parafac
from mpl_toolkits.axes_grid1 import make_axes_locatable
import tensorly as tl
from tensorly.decomposition import non_negative_parafac
from tensorly.decomposition import non_negative_tucker
from tensorly.decomposition import robust_pca
from tensorly.decomposition import parafac
from sklearn.decomposition import NMF
from google.colab import files
from pylab import *
from scipy.optimize import curve_fit
from matplotlib import colors
from scipy.cluster.vq import whiten, kmeans, vq
import scipy
from tensorly import metrics
import torch
from scipy.ndimage.interpolation import rotate

In [ ]:
!pip install SGMLParseError


In [ ]:
from matplotlib.colors import ListedColormap
from matplotlib.colors import BoundaryNorm

# Load and Calibrate Data 

Let's load the data. Need to get it from google drive..

We also calibrate it given the sensitivity of the LDV, to convert it to pm.

In [ ]:
# %%shell
#Data available upon request.


In [ ]:
folder = r'.'
file_name = #h5 file name
h5_f = h5py.File(os.path.join(folder, file_name), 'r+')
usid.hdf_utils.print_tree(h5_f)

In [ ]:
raw_data = usid.hdf_utils.find_dataset(h5_f, 'Raw_Data')[-1]
spec_vals = h5_f['Measurement_000/Channel_000/Raw_Data-SHO_Fit_000/Spectroscopic_Values']
spec_vals_nosho = h5_f['Measurement_000/Channel_000/Spectroscopic_Values']
spec_inds = h5_f['Measurement_000/Channel_000/Raw_Data-SHO_Fit_000/Spectroscopic_Indices']
pos_inds = h5_f['Measurement_000/Channel_000/Position_Indices']
pos_vals = h5_f['Measurement_000/Channel_000/Position_Values']

#Convert to proper units
num_freq_steps = len(np.unique(h5_f['Measurement_000/Channel_000/Spectroscopic_Values'][0,:]))
ldv_correction_factor = 20.0 / (2*np.pi*(spec_vals_nosho[0,num_freq_steps//2]))*1000000    ### produces nm/V correction factor i.e. sensitivity factor
data_mat = raw_data[:,num_freq_steps//2::num_freq_steps]

#Get the time vector
time_per_step = usid.hdf_utils.get_attributes(h5_f['Measurement_000'])['BE_pulse_duration_[s]']

# Correct the Phase and plot averages

There's always a pesky phase offset to deal with. Let's just plot the histogram and shift it over so it is centered around 0 and pi radians...

In [ ]:
amp = np.abs(data_mat[:])* ldv_correction_factor*1000
phase = np.angle(data_mat[:])

In [ ]:
plt.figure()
plt.hist(phase.ravel(), bins = 100);
phase_offset = 1.41
phase_new = -1*(phase - phase_offset)
plt.figure()
plt.hist(phase_new.ravel(), bins = 100);

In [ ]:
#Plot average relaxation spectra (blue) and voltage waveform (red)

step = np.arange(spec_vals.shape[1])*time_per_step


fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
ax1.tick_params(labelsize = 16)
plt.xlim(-0.5,14.5)
plt.ylim(7,45)
line1 = ax1.plot(step, amp.mean(axis=0), color = 'blue')
ax2 = fig1.add_subplot(111, sharex=ax1, frameon=False)
line2 = ax2.plot(step,spec_vals[1,:], color = 'red')
ax2.yaxis.tick_right()
plt.ylim(-20,20)
ax2.tick_params(labelsize = 16)
ax2.yaxis.set_label_position("right")
plt.tight_layout()


# Build the Piezoresponse Matrices

In [ ]:
a = np.diff(spec_vals[1,:])
num_DC_voltages = len(a[a<0])
dc_vec = spec_vals[1,:][spec_vals[2,:]==1]
dc_vec_unique=[]
dc_vec_unique.append(dc_vec[0]) #Start with the initial voltage

for ind in range(1,len(dc_vec)):
    if dc_vec[ind] != dc_vec[ind-1]:
        dc_vec_unique.append(dc_vec[ind])

dc_vec_unique_arr = np.zeros(len(dc_vec_unique)+1)
dc_vec_unique_arr[:len(dc_vec_unique_arr)//2] = dc_vec_unique[:len(dc_vec_unique_arr)//2]
dc_vec_unique_arr[len(dc_vec_unique_arr)//2] = dc_vec_unique[(len(dc_vec_unique_arr)//2) -1]
dc_vec_unique_arr[len(dc_vec_unique_arr)//2 +1:] = dc_vec_unique[len(dc_vec_unique_arr)//2 :]

num_dc_voltages = len(dc_vec_unique_arr)

PR_mat = amp*np.cos(phase_new)


pos_dim_sizes = [pos_inds[:,0].max()+1, pos_inds[:,1].max()+1]

PR_mat_OF = PR_mat[:,spec_vals[2,:]==0]
# PR_mat_OF = PR_mat_OF - np.min(PR_mat_OF) +100
PR_mat_IF = PR_mat[:,spec_vals[2,:]==1]

#PR_mat_ndim = PR_mat.reshape(pos_dim_sizes[0], pos_dim_sizes[1],-1,len(np.unique(spec_vals[0,:])))

PR_mat_OF_ndim = PR_mat_OF.reshape(pos_dim_sizes[0], pos_dim_sizes[1],num_dc_voltages,-1)
PR_mat_IF_ndim = PR_mat_IF.reshape(pos_dim_sizes[0], pos_dim_sizes[1],num_dc_voltages,-1)


phase_mat_OF = phase_new[:,spec_vals[2,:]==0]
phase_mat_OF_ndim = phase_mat_OF.reshape(pos_dim_sizes[0], pos_dim_sizes[1],num_dc_voltages,-1)

amp_mat_OF = amp[:,spec_vals[2,:]==0]
amp_mat_OF_ndim = amp_mat_OF.reshape(pos_dim_sizes[0], pos_dim_sizes[1],num_dc_voltages,-1)

time_vec_OF = np.arange(0, PR_mat_OF_ndim.shape[-1])*time_per_step
time_vec_IF = np.arange(0, PR_mat_IF_ndim.shape[-1])*time_per_step

# Plot spatial maps

In [ ]:
#Let's plot spatially

PR_mat_OF_ndim = PR_mat_OF.reshape(pos_dim_sizes[0], pos_dim_sizes[1],num_dc_voltages,-1)
amp_mat_OF_ndim  = amp_mat_OF.reshape(pos_dim_sizes[0], pos_dim_sizes[1],num_dc_voltages,-1)
#PR_mat_IF_ndim = PR_mat_IF.reshape(pos_dim_sizes[0], pos_dim_sizes[1],num_dc_voltages,-1)
fig, axes = plt.subplots(nrows=4,ncols=4, figsize = (12,12))

for ind, ax in enumerate(axes.flat):
    
    ax.imshow(phase_mat_OF_ndim[:,:,ind,0])
    ax.set_title('Phase, V={}V t=0'.format(np.round(dc_vec_unique_arr[ind],2)))
    ax.axis('off')
fig.tight_layout()

fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle('Phase Images Off-field', fontsize =18)

fig, axes = plt.subplots(nrows=4,ncols=4, figsize = (12,12))

for ind, ax in enumerate(axes.flat):
    
    ax.imshow(PR_mat_OF_ndim[:,:,ind,0])
    ax.set_title('PR, V={}V t=0'.format(np.round(dc_vec_unique_arr[ind],2)))
#     ax.axis('off')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle('PR Images Off-field', fontsize =18)


In [ ]:
#Plots spatial defined line of piezoresponse vs voltage in a range of relaxation times 

derp = []
start_xpix = 50
yrow_pix = 54
line_length = 10
num_time_step = 10

r_data = PR_mat_OF_ndim

for jj in range(line_length):   #chooses the pixels to examine - swith x and y of PR mat to change to vertical
  
  derp.append(r_data[start_xpix+jj,yrow_pix,:,:num_time_step])

derp = np.array(derp)

colors = plt.cm.get_cmap('plasma',PR_mat_OF_ndim.shape[2])

n_linex = np.arange(start_xpix, start_xpix+jj,1)  
n_liney = np.full(jj,yrow_pix)

plt.figure()
plt.imshow(r_data[:,:,8,0], clim = (-50,70), cmap = 'viridis')
plt.colorbar()
plt.axis('off')

spotx = [26,13,38,55]
spoty = [50,13,39,54]
imag_color =  ['blue' , 'red' , 'magenta', 'orange']
name  = ['c-domain 1', 'a-domain', 'a-c domain wall' , 'c-domain 2']

plt.figure()

spot = 3
tlen =84
for ii in range(0,tlen,4):

  plt.plot(dc_vec_unique_arr[:],r_data[spotx[spot],spoty[spot],:,ii], color = colors(ii/tlen))

plt.legend(frameon = False)
plt.ylim(-70,70)
plt.ylabel('Piezoresponse [pm]', size = 18)
plt.xlabel('Voltage [V]', size = 18)
plt.xticks(size = 16)
plt.yticks(size = 16)
plt.tight_layout()

# Perform NTF

Here, we perform non-negative tensor factorization. We start with the off-field full piezoresponse

In [ ]:
num_ranks = 4

data = tl.tensor(amp_mat_OF_ndim[:,:,:,:].reshape(amp.shape[0],amp_mat_OF_ndim.shape[2],amp_mat_OF_ndim.shape[3] ))

weights, factors = non_negative_parafac(data, rank=num_ranks,n_iter_max = 1000, tol = 100E-30000000000000000000)
fig, axes = plt.subplots(nrows = num_ranks, ncols = 3, figsize = (15,15))

def func(x, a, c, d, f, g):
    return a*np.exp(-c*x) + f*np.exp(-g*x) + d 

popt_pr = []
pcov_pr = []
  
for row in range(num_ranks):
    im = axes[row,0].imshow(np.sqrt(factors[0][:,row].reshape(pos_dim_sizes)/(np.max(factors[0][:,row]))),clim = (0,1))  
    
    axes[row,0].axis('off')
    divider = make_axes_locatable(axes[row,0])
    cax = divider.append_axes("right", size="5%", pad=0.08) #space for colorbar
    plt.colorbar(im, cax = cax)
    x = time_vec_OF[1:]  
    y = factors[2][1:,row]
    popt, pcov = curve_fit(func, x, y, p0=(1e-1, 10, 0,1,1),maxfev = 100000, absolute_sigma = True)
    popt_pr.append(popt)
    pcov_pr.append(pcov)
    yy=func(x,*popt)
    axes[row,1].plot(dc_vec_unique_arr, factors[1][:,row],color = 'green')
    axes[row,1].set_xlabel('Voltage [V]', size =20)
    axes[row,1].set_ylabel('Scaling Factor', size =20)
    axes[row,1].set_ylim(0,0.75)
    axes[row,1].tick_params(labelsize=16)
    axes[row,2].plot(time_vec_OF, factors[2][:,row], 'o', color = 'red', label = 'Raw')
    axes[row,2].plot(x,yy, '--', color = 'black', label = 'Two Exp Fit')
    axes[row,2].set_xlabel('Time [s]',size =20)
    axes[row,2].set_ylabel('PR Amp [a.u.]', size =20)
    axes[row,2].tick_params(labelsize=16)
    axes[row,2].legend(frameon= False)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle('All voltages relaxations Off-field', fontsize =18)

In [ ]:
def func(x, a, c, d, f, g):
    return a*np.exp(-c*x) + f*np.exp(-g*x) + d 
fig, axes = plt.subplots(nrows = num_ranks, ncols = 3, figsize = (15,15))
popt_pr = []
pcov_pr = []
  
for row in range(num_ranks):
    im = axes[row,0].imshow((factors[0][:,row].reshape(pos_dim_sizes)/(np.max(factors[0][:,row]))),clim = (0,1))  
    axes[row,0].axis('off')
    divider = make_axes_locatable(axes[row,0])
    cax = divider.append_axes("right", size="5%", pad=0.08) #space for colorbar
    plt.colorbar(im, cax = cax)
    x = time_vec_OF[1:]  
    y = factors[2][1:,row]
    popt, pcov = curve_fit(func, x, y, p0=(1e-1, 10, 0,1,1),maxfev = 100000, absolute_sigma = True)
    popt_pr.append(popt)
    pcov_pr.append(pcov)
    yy=func(x,*popt)
    axes[row,1].plot(dc_vec_unique_arr, factors[1][:,row],color = 'green')
    axes[row,1].set_xlabel('Voltage [V]', size =20)
    axes[row,1].set_ylabel('Scaling Factor', size =20)
    axes[row,1].set_ylim(0,0.75)
    axes[row,1].tick_params(labelsize=16)
    axes[row,2].plot(time_vec_OF, factors[2][:,row], 'o', color = 'red', label = 'Raw')
    axes[row,2].plot(x,yy, '--', color = 'black', label = 'Two Exp Fit')
    axes[row,2].set_xlabel('Time [s]',size =20)
    axes[row,2].set_ylabel('PR Amp [a.u.]', size =20)
    axes[row,2].tick_params(labelsize=16)
    axes[row,2].legend(frameon= False)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])

In [ ]:
#Look at different exponential fits for NTF analysis
row = 3

def func_1(x, a, c, d):
    return a*np.exp(-c*x) + d #f*np.exp(-g*x)

def func_2(x, a, c, d,f,g):
    return a*np.exp(-c*x) + d + f*np.exp(-g*x)

def func_3(x, a, c, d):
    return a*np.exp(-x**c)+ d 

x = time_vec_OF[1:]  
y = factors[2][1:,row]

popt, pcov = curve_fit(func_1, x, y, p0=(1e-1, 10, 0),maxfev = 100000, absolute_sigma = True) #p0=(1e-1, 10, 0,1,1)
yy=func_1(x,*popt)

popt2, pcov2 = curve_fit(func_2, x, y, p0=(1e-1, 10, 0,1,1),maxfev = 100000, absolute_sigma = True) #p0=(1e-1, 10, 0,1,1)
yy2=func_2(x,*popt2)

popt3, pcov3 = curve_fit(func_3, x, y, p0=(-1.63788657e+00,  7.53257186e-04,  6.78572883e-01),maxfev = 100000, absolute_sigma = True) #p0=(1e-1, 10, 0,1,1)
yy3=func_3(x,*popt3)

perr = np.sqrt(np.diag(pcov2))

ss_res = np.sum((y - yy2) ** 2)
ss_tot = np.sum((y - np.mean(y)) ** 2)
r2 = 1 - (ss_res / ss_tot)
print('R2 value for double exp: ' +str(r2))
ss_res = np.sum((y - yy) ** 2)
ss_tot = np.sum((y - np.mean(y)) ** 2)
r2 = 1 - (ss_res / ss_tot)
print('R2 value for single exp: ' +str(r2))

print('Single exp t1: ' + str(1/popt[1]))

print('Double exp t1: ' + str(1/popt2[1])+ ' and t2: ' + str(1/popt2[4]))

fig = plt.figure(figsize=(12,6))
grid = plt.GridSpec(6,12, wspace=0.4, hspace=4)

img1 = fig.add_subplot(grid[:6, :6])
img1.imshow(factors[0][:,row].reshape(pos_dim_sizes)/(np.max(factors[0][:,row])))
img1.axis('off')

img2 = fig.add_subplot(grid[0:3,7:])
img2.plot(time_vec_OF, factors[2][:,row]*10, 'o', color = 'red', label = 'Raw')
img2.plot(x,yy2*10, '--', color = 'black', label = 'Dual Exp Fit')
img2.tick_params(labelsize=16)
img2.legend(frameon= False, loc =4)

img3 = fig.add_subplot(grid[3:,7:])
img3.plot(dc_vec_unique_arr, factors[1][:,row],color = 'green')
img3.tick_params(labelsize=16)
img3.set_ylim(0,0.8)

In [ ]:
#Calculate relaxation line averages, vertically

from scipy.ndimage import rotate
mat_avg1 = []

for ii in range(128):
  rotated_mat1 = rotate(amp_mat_OF_ndim[20:,:40,1,ii],-5)
  rotated_mat1 = rotated_mat1[4:-5, 4:-3]
  mat_avg1.append(rotated_mat1.mean(axis=0))

In [ ]:
plt.figure()
plt.imshow(rotated_mat1);

plt.figure()
plt.plot(mat_avg1[127,:], '-o');

a = np.arange(0,128)*time_per_step

plt.figure(figsize=(5,4))
plt.plot(a[1:],mat_avg1[1:,12:17],'-o', markersize = 3);
plt.xlim(-0.03,1)
plt.ylabel('PR Amplitude [pm]', fontsize = 18)
plt.xlabel('Time [s]',fontsize = 18)
plt.xticks(size = 16)
plt.yticks(size = 16)
plt.tight_layout()

In [ ]:
row=2
plt.figure()
mat = factors[0][:,row].reshape(pos_dim_sizes)/(np.max(factors[0][:,row]))
plt.imshow(mat[25:,:40])

mat_cropped = mat[25:,:44]
mat_cropped = rotate(mat[20:,:45], angle = -6)[5:40,3:39]
mat_averaged = mat_cropped.mean(axis=0)

plt.figure()
plt.imshow(mat_cropped,clim = (0.0,0.6))
plt.colorbar()
plt.axis('off')
plt.savefig('zoom_abd3.pdf', dpi =600)
files.download("zoom_abd3.pdf")

spat_step = 1200/60
step_ = np.linspace(0,mat_averaged.shape[0],mat_averaged.shape[0])*spat_step

plt.figure()
plt.plot(step_,mat_averaged, label = 'Averaged Profile', color = 'red', linewidth = 3)
plt.xlabel('X Position [nm]',fontsize = 18)
plt.ylabel('Comp. 3 Abund. [a.u.]', fontsize = 18,labelpad = 0)
plt.xticks(size = 14)
plt.yticks(size = 14)
plt.legend(frameon = False)
plt.tight_layout()

In [ ]:
#NTF error

X = tl.tensor(data)
norm_tensor = tl.norm(X, 2)
rec_error = tl.norm(X - tl.kruskal_to_tensor([weights,factors]), 2) / norm_tensor

print('NTF Error: ' +str(rec_error*100) +'%')

In [ ]:
#Calculates relaxation times for user defined row in NTF results
row = 0

def func(x, a, c, d, f, g):
    return a*np.exp(-c*x) + f*np.exp(-g*x) + d 
  
def fun_stretch(x,a,b,d):
    return a*np.exp(-x**b) +d

x = time_vec_OF[1:]  
y = factors[2][1:,row]
popt, pcov = curve_fit(fun_stretch, x, y, p0=(-1e-3,2.1,0),maxfev = 100000, absolute_sigma = True)
yy=fun_stretch(x,*popt)

popt1, pcov1 = curve_fit(func, x, y,p0=(1e-1, 10, 0,1,1),maxfev = 100000, absolute_sigma = True)
yy1=func(x,*popt1)

plt.figure()
plt.plot(time_vec_OF, factors[2][:,row], 'o', color = 'red', label = 'Raw')
# plt.plot(x,yy*100, '-', color = 'black', label = 'Two Exp Fit')
plt.plot(x,yy1, '--', color = 'black', label = 'Two Exp Fit')

t1 = 1/popt1[1]

t2 = 1/popt1[4]

print('t1 ' + str(t1) +'and t2 ' + str(t2))

In [ ]:
#Plots difference map of -15V and +15V

cut =15
  
PR_diff = (PR_mat_OF_ndim[:,:,0,0] - PR_mat_OF_ndim[:,:,8,0]) - (PR_mat_OF_ndim[:,:,0,127] - PR_mat_OF_ndim[:,:,8,127]) #plot as a function of time

fig, axes = plt.subplots(nrows=2,ncols=3, figsize = (16,10))

im = axes[0,1].imshow(-1*amp_mat_OF_ndim[cut:,cut:,0,0] + amp_mat_OF_ndim[cut:,cut:,8,0])
divider = make_axes_locatable(axes[0,1])
cax = divider.append_axes("right", size="5%", pad=0.08) #space for colorbar
plt.colorbar(im, cax = cax)
axes[0,1].axis('off')

im = axes[0,2].imshow((-1*phase_mat_OF_ndim[cut:,cut:,0,127] + phase_mat_OF_ndim[cut:,cut:,8,127])*180/np.pi, clim = (10,-190))
divider = make_axes_locatable(axes[0,2])
cax = divider.append_axes("right", size="5%", pad=0.08) #space for colorbar
plt.colorbar(im, cax = cax)
axes[0,2].axis('off')

im = axes[0,0].imshow(PR_mat_OF_ndim[cut:,cut:,0,0], clim = (-100,20))
divider = make_axes_locatable(axes[0,0])
cax = divider.append_axes("right", size="5%", pad=0.08) #space for colorbar
plt.colorbar(im, cax = cax)
axes[0,0].axis('off')

for row in range(3):
  derp =factors[0][:,row].reshape(pos_dim_sizes)  
  im = axes[1,row].imshow(derp[cut:,cut:]/(np.max(derp[cut:,cut:])),clim = (0,1))
  if row ==0:
      im = axes[1,row].imshow(derp[cut:,cut:]/(np.max(derp[cut:,cut:])),clim = (0,1))
  axes[1,row].axis('off')  
  divider = make_axes_locatable(axes[1,row])
  cax = divider.append_axes("right", size="5%", pad=0.08) #space for colorbar
  plt.colorbar(im, cax = cax)

In [ ]:
PR_mat_OF_ndim.shape
data1 = tl.tensor(PR_mat_OF_ndim[:,:,:,:].reshape(PR_mat.shape[0],PR_mat_OF_ndim.shape[2],PR_mat_OF_ndim.shape[3] ))
data1 = data1.reshape(3600,16*128)
data1.shape

In [ ]:
#KMEANS cluster data - take average of cluster locations

clusters = 5

data1 = tl.tensor(PR_mat_OF_ndim[:,:,:,:].reshape(PR_mat.shape[0],PR_mat_OF_ndim.shape[2]*PR_mat_OF_ndim.shape[3] ))
data_res = tl.tensor(PR_mat_OF_ndim[:,:,:,:].reshape(PR_mat.shape[0],PR_mat_OF_ndim.shape[2],PR_mat_OF_ndim.shape[3] ))
data1 = scipy.cluster.vq.whiten(data1[:,:]) #data1[1] = 6 corresponds to 15V pulse
centroids,_  = scipy.cluster.vq.kmeans(data1,clusters,iter = 100)

clx,_ = vq(data1,centroids)

derp = np.reshape(clx,(60,60))

cmap = ListedColormap(['blue', 'orange', '0.8', 'green','red'])
cmaplist = [cmap(i) for i in range(cmap.N)]
cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
bounds = np.linspace(0,5,6)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

plt.figure()
plt.imshow(derp, cmap = cmap, norm = norm)
plt.axis('off')
plt.colorbar(cmap =cmap,boundaries = bounds, norm = norm)

In [ ]:
#Plot average of individual kmeans clusters
stoppt = 128 #number of time loops in graph

data_res = tl.tensor(PR_mat_OF_ndim[:,:,:,:].reshape(PR_mat.shape[0],PR_mat_OF_ndim.shape[2],PR_mat_OF_ndim.shape[3] ))

comp0 = []
comp1 = []
comp2 = []
comp3 = []
comp4 = []

for iii in range(data_res.shape[0]):
  if clx[iii] == 0:
    comp0.append(data_res[iii,:,:])
  if clx[iii] == 1:
    comp1.append(data_res[iii,:,:])
  if clx[iii] == 2:
    comp2.append(data_res[iii,:,:])
  if clx[iii] == 3:
    comp3.append(data_res[iii,:,:])
  if clx[iii] == 4:
    comp4.append(data_res[iii,:,:])

compnum = [comp0,comp1,comp2,comp3,comp4]

for ij in range(5):
  comp2avg = []
  comp2r = np.array(compnum[ij])

  for ii in range(comp2r.shape[1]):
    for jj in range(comp2r.shape[2]):
      comp2avg.append(np.mean(comp2r[:,ii,jj]))

  comp2avg = np.reshape(np.array(comp2avg), (16,128))
  colors = plt.cm.get_cmap('viridis',128)


  plt.figure(figsize=(5,4))
  for aa in range(stoppt):
    plt.plot(dc_vec_unique_arr,comp2avg[:,aa], color = colors(aa/(stoppt*1.08)))
  plt.ylim(-70,40)
  plt.ylabel('Piezoresponse [pm]', size = 18)
  plt.xlabel('Voltage [V]', size = 18)
  plt.xticks(size = 16)
  plt.yticks(size = 16)
  plt.tight_layout()


In [ ]:
#Plots spatial defined line of piezoresponse vs voltage in a range of relaxation times 

derp1 = []
start_xpix_i = 2
yrow_pix_i = 49
line_length = 8
num_time_step = 128 #essentially number of loops to look at
num_time_step_interval = 8

r_data = PR_mat_OF_ndim
derp = np.reshape(clx,(60,60))

c = 'blue'
a = 'orange'
b = '0.8'
d = 'green'

cmap = ListedColormap(['0.8','orangered' , 'blue', 'green','orange'])
cmaplist = [cmap(i) for i in range(cmap.N)]
cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
bounds = np.linspace(0,5,6)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

fig, axes = plt.subplots(nrows=1,ncols=8, figsize = (20,4)) # Change subplot size with line size
axes = axes.flatten()

for jj in range(line_length):   #chooses the pixels to examine - swith x and y of PR mat to change to vertical
  derp1.append(r_data[yrow_pix_i,start_xpix_i+jj,:,:num_time_step]) # saves a subset of PR_mat_OF_ndim as derp1 - its the line
derp1 = np.array(derp1)
im = ax.imshow(np.random.random((16, 16)), cmap='viridis',vmin=0, vmax=0.77)
colors = plt.cm.get_cmap('viridis',PR_mat_OF_ndim.shape[2])

stop = int(num_time_step/num_time_step_interval)

for ii in range(derp1.shape[0]): #plots the hystersis loops of the line - ii steps through pixel, z steps through time
    for z in range(stop):
      axes[ii].plot(dc_vec_unique_arr[:],derp1[ii,:,z*num_time_step_interval] , color = colors(z))     
    axes[ii].set_xlabel('Voltage [V]')
    plt.title('Pixel Number ' + str(start_xpix_i+ ii))
fig.tight_layout()
fig.colorbar(im,ax=axes.ravel().tolist(), shrink=0.8)

n_linex_i = np.arange(start_xpix_i, start_xpix_i+jj,1)  
n_liney_i = np.full(jj,yrow_pix_i)

plt.figure()
plt.imshow(derp, cmap = cmap, norm = norm)
plt.plot(n_linex_i,n_liney_i, color='white', linewidth = 2)
plt.axis('off')
plt.colorbar(cmap =cmap,boundaries = bounds, norm = norm)

In [ ]:
#cluster all data

clusters = 4

data1 = tl.tensor(PR_mat_OF_ndim[:,:,:,:].reshape(PR_mat.shape[0],PR_mat_OF_ndim.shape[2],PR_mat_OF_ndim.shape[3] ))
data_res = data1
fig, axes = plt.subplots(nrows=1,ncols=5, figsize = (20,20))
axes = axes.flatten()

for volt_step in range(5):
  data1 = scipy.cluster.vq.whiten(data_res[:,volt_step*2,:])
  centroids,_  = scipy.cluster.vq.kmeans(data1,clusters,iter = 100)

  clx,_ = vq(data1,centroids)

  derp = np.reshape(clx,(60,60))

  im = axes[volt_step].imshow(derp, cmap = 'viridis')
  axes[volt_step].axis('off')

In [ ]:
comp2avg = []
comp2r = np.array(comp4)

for ii in range(comp2r.shape[1]):
  for jj in range(comp2r.shape[2]):
    comp2avg.append(np.mean(comp2r[:,ii,jj]))

comp2avg = np.array(comp2avg)

comp2avg = np.reshape(comp2avg, (16,128))

colors = plt.cm.get_cmap('plasma',128)

stoppt = 128

plt.figure()
for aa in range(stoppt):
  plt.plot(dc_vec_unique_arr,comp2avg[:,aa], color = colors(aa/(stoppt)))
plt.ylim(-70,70)
plt.ylabel('Piezoresponse [pm]', size = 18)
plt.xlabel('Voltage [V]', size = 18)
plt.xticks(size = 16)
plt.yticks(size = 16)
plt.tight_layout()

In [ ]:
def func(x, a, c, d, f, g):
    return a*np.exp(-c*x) + f*np.exp(-g*x) + d 

row = 0
x = time_vec_OF[2:]  
y = factors[2][2:,row]
popt, pcov = curve_fit(func, x, y, p0=(1e-1, 10, 0,1,1),maxfev = 100000, absolute_sigma = True)
yy=func(x,*popt)

plt.figure()
plt.plot(time_vec_OF, factors[2][:,row]*100)
plt.plot(x,yy*100, '--', color = 'black', label = 'Two Exp Fit')

We now try the in-field full piezoresponse

In [ ]:
num_ranks = 3

data = tl.tensor(PR_mat_IF_ndim[:,:,:,:].reshape(PR_mat.shape[0],PR_mat_IF_ndim.shape[2],PR_mat_IF_ndim.shape[3] ))
weights, factors = non_negative_parafac(data, rank=num_ranks)
fig, axes = plt.subplots(nrows = num_ranks, ncols = 3, figsize = (10,10))

for row in range(num_ranks):
    im = axes[row,0].imshow(factors[0][:,row].reshape(pos_dim_sizes))
    divider = make_axes_locatable(axes[row,0])
    cax = divider.append_axes("right", size="5%", pad=0.08) #space for colorbar
    plt.colorbar(im, cax = cax)
    axes[row,1].plot(dc_vec_unique_arr, factors[1][:,row])
    axes[row,1].set_title('Voltage')
    axes[row,2].plot(time_vec_IF, factors[2][:,row])
    axes[row,2].set_title('Time')
fig.tight_layout()

fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle('All voltages relaxations In-field', fontsize = 18)

Now let's look at the raw piezoreponse amplitude off-field

In [ ]:
num_ranks = 4

data = tl.tensor(amp_mat_OF_ndim[:,:,:,:].reshape(PR_mat.shape[0],PR_mat_OF_ndim.shape[2],PR_mat_OF_ndim.shape[3] ))
weights, factors = non_negative_parafac(data, rank=num_ranks)
fig, axes = plt.subplots(nrows = num_ranks, ncols = 3, figsize = (10,10))

for row in range(num_ranks):
    im = axes[row,0].imshow(factors[0][:,row].reshape(pos_dim_sizes))
    divider = make_axes_locatable(axes[row,0])
    cax = divider.append_axes("right", size="5%", pad=0.08) #space for colorbar
    plt.colorbar(im, cax = cax)
    axes[row,1].plot(dc_vec_unique_arr, factors[1][:,row])
    axes[row,1].set_title('Voltage')
    axes[row,2].plot(time_vec_OF, factors[2][:,row])
    axes[row,2].set_title('Time')
fig.tight_layout()

fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle('All voltages Amp relaxations Off-field', fontsize = 18)

# Plot hysteresis loops as a function of time, voltage and space

To see the data a bit better, let us also just plot all the loops on the grid. **This may take some time to plot!**

In [ ]:
#Let's plot hysteresis loops as a function of time

PR_mat_OF_ndim = PR_mat_OF.reshape(pos_dim_sizes[0], pos_dim_sizes[1],num_dc_voltages,-1)
fig, axes = plt.subplots(nrows=30,ncols=30, figsize = (30,30))

colors = plt.cm.get_cmap('viridis',PR_mat_OF_ndim.shape[-1])

indices_col = np.tile(np.arange(0,pos_dim_sizes[0],step = 2), pos_dim_sizes[1])
indices_row = np.repeat(np.arange(0,pos_dim_sizes[0],step = 2), pos_dim_sizes[1])

for ind, ax in enumerate(axes.flat):
    pix_index_col = indices_col[ind]
    pix_index_row = indices_row[ind]
    for i in range(PR_mat_OF_ndim.shape[-1]-110):
        ax.plot(dc_vec_unique_arr,PR_mat_OF_ndim[pix_index_col,pix_index_row,:,i], color = colors(i))
        ax.axis('off')

# Perform NMF

Finally, do some NMF for comparison purposes

In [ ]:
#Now let's do NMF instead
amp_mat_OF_flat = amp_mat_OF_ndim.reshape(np.prod(pos_dim_sizes)*amp_mat_OF_ndim.shape[-2],amp_mat_OF_ndim.shape[-1])
num_comps = 4
nmf = NMF(n_components=num_comps, max_iter=1000)

nmf.fit(amp_mat_OF_flat)
components = nmf.components_
projections =nmf.fit_transform(amp_mat_OF_flat)
projections_reshaped = projections.reshape(pos_dim_sizes[0],pos_dim_sizes[1],-1, num_comps)

In [ ]:
from sklearn.decomposition.nmf import _beta_divergence

nmf.reconstruction_err_

In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=4, figsize = (20,4))
a = np.arange(0,128)*time_per_step

for ind, ax in enumerate(axes.flat):
    ax.plot(a,components[ind,:], '-o', markersize = 4, color = 'red')
    ax.set_xlabel('Time', size = 16)
    ax.tick_params(labelsize=16)
fig.tight_layout()

fig, axes = plt.subplots(nrows=1,ncols=4, figsize = (18,4))

for ind, ax in enumerate(axes.flat):
    ax.imshow(projections_reshaped[:,:,0,ind])
    ax.axis('off')   
fig.tight_layout()

fig, axes = plt.subplots(nrows=1,ncols=4,figsize = (18,4))

for ind, ax in enumerate(axes.flat):
    ax.imshow(projections_reshaped[:,:,7,ind])
    ax.axis('off')   
fig.tight_layout()
